In [1]:
!pip install unidecode

The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Import TensorFlow >= 1.9 and enable eager execution
import tensorflow as tf

# Note: Once you enable eager execution, it cannot be disabled. 
tf.enable_eager_execution()

import numpy as np
import re
import random
import unidecode
import time

In [3]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/yashkatariya/shakespeare.txt')
path_to_file = 'poem_corpus.txt'

In [4]:
text = unidecode.unidecode(open(path_to_file).read())
print(text[:1000])

          CHRISTMAS NIGHT.


    Be peace on earth, good will to men;
      And let this now our carol be:
      If on the land, or on the sea,
    We still will sing the glad refrain;
      And in the closing light of day
      Good words of peace and cheer will say.

    The Babe that in the manger born
      Has risen high above the star,
      To judge in peace, or judge in war,
    To judge at night or judge at morn.
      The star that told us of his birth
      Has given us joy and lasting mirth.

    The Man that suffered on the tree
      Is risen high above all men;
      Then swell the glad refrain again--
    He died for me, He died for thee:
      Then peace be ever on the earth
      To one and all of human birth.




          FALLING OF THE APPLE TREE.


    The apple tree has fallen, now--
      The axe has laid it low;
    The blossoms sparkled ere it fell,
      But now they wither so.

    Its shade we now shall seek in vain--
      The spot we loved so well
    Has

In [5]:
# unique contains all the unique characters in the file
unique = sorted(set(text))

# creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(unique)}
idx2char = {i:u for i, u in enumerate(unique)}

In [6]:
# setting the maximum length sentence we want for a single input in characters
max_length = 100

# length of the vocabulary in chars
vocab_size = len(unique)

# the embedding dimension 
embedding_dim = 256

# number of RNN (here GRU) units
units = 1024

# batch size 
BATCH_SIZE = 64

# buffer size to shuffle our dataset
BUFFER_SIZE = 10000

In [7]:
input_text = []
target_text = []

for f in range(0, len(text)-max_length, max_length):
    inps = text[f:f+max_length]
    targ = text[f+1:f+1+max_length]

    input_text.append([char2idx[i] for i in inps])
    target_text.append([char2idx[t] for t in targ])
    
print (np.array(input_text).shape)
print (np.array(target_text).shape)

(15820, 100)
(15820, 100)


In [8]:
np.array(input_text)[:3]

array([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 27, 32, 42, 33, 43, 44, 37,
        25, 43,  1, 38, 33, 31, 32, 44, 11,  0,  0,  0,  1,  1,  1,  1, 26,
        58,  1, 69, 58, 54, 56, 58,  1, 68, 67,  1, 58, 54, 71, 73, 61,  9,
         1, 60, 68, 68, 57,  1, 76, 62, 65, 65,  1, 73, 68,  1, 66, 58, 67,
        23,  0,  1,  1,  1,  1,  1,  1, 25, 67, 57,  1, 65, 58, 73,  1, 73,
        61, 62, 72,  1, 67, 68, 76,  1, 68, 74, 71,  1, 56, 54, 71],
       [68, 65,  1, 55, 58, 22,  0,  1,  1,  1,  1,  1,  1, 33, 59,  1, 68,
        67,  1, 73, 61, 58,  1, 65, 54, 67, 57,  9,  1, 68, 71,  1, 68, 67,
         1, 73, 61, 58,  1, 72, 58, 54,  9,  0,  1,  1,  1,  1, 47, 58,  1,
        72, 73, 62, 65, 65,  1, 76, 62, 65, 65,  1, 72, 62, 67, 60,  1, 73,
        61, 58,  1, 60, 65, 54, 57,  1, 71, 58, 59, 71, 54, 62, 67, 23,  0,
         1,  1,  1,  1,  1,  1, 25, 67, 57,  1, 62, 67,  1, 73, 61],
       [58,  1, 56, 65, 68, 72, 62, 67, 60,  1, 65, 62, 60, 61, 73,  1, 68,
        59,  1, 57, 54, 78

In [9]:
np.array(target_text)[0][0]

1

In [37]:
idx2char[38]

'N'

In [11]:
''.join(idx2char[target_text[0][i]] for i in range(len(target_text[0])))

'         CHRISTMAS NIGHT.\n\n\n    Be peace on earth, good will to men;\n      And let this now our caro'

In [12]:
''.join(idx2char[input_text[0][i]] for i in range(len(input_text[0])))

'          CHRISTMAS NIGHT.\n\n\n    Be peace on earth, good will to men;\n      And let this now our car'

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((input_text, target_text)).shuffle(BUFFER_SIZE)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.


In [23]:
class Model(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, units, batch_size):
    super(Model, self).__init__()
    self.units = units
    self.batch_sz = batch_size

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    if tf.test.is_gpu_available():
      print('GPU available')
      self.gru = tf.keras.layers.CuDNNGRU(self.units, 
                                          return_sequences=True, 
                                          return_state=True, 
                                          recurrent_initializer='glorot_uniform')
    else:
      print('GPU available')  
      self.gru = tf.keras.layers.GRU(self.units, 
                                     return_sequences=True, 
                                     return_state=True, 
                                     recurrent_activation='sigmoid', 
                                     recurrent_initializer='glorot_uniform')

    self.fc = tf.keras.layers.Dense(vocab_size)
        
  def call(self, x, hidden):
    x = self.embedding(x)

    # output shape == (batch_size, max_length, hidden_size) 
    # states shape == (batch_size, hidden_size)

    # states variable to preserve the state of the model
    # this will be used to pass at every step to the model while training
    output, states = self.gru(x, initial_state=hidden)


    # reshaping the output so that we can pass it to the Dense layer
    # after reshaping the shape is (batch_size * max_length, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # The dense layer will output predictions for every time_steps(max_length)
    # output shape after the dense layer == (max_length * batch_size, vocab_size)
    x = self.fc(output)

    return x, states

In [24]:
model = Model(vocab_size, embedding_dim, units, BATCH_SIZE)

GPU available


In [25]:
optimizer = tf.train.AdamOptimizer()

# using sparse_softmax_cross_entropy so that we don't have to create one-hot vectors
def loss_function(real, preds):
    return tf.losses.sparse_softmax_cross_entropy(labels=real, logits=preds)

In [ ]:
# Training step

EPOCHS = 30

for epoch in range(EPOCHS):
    start = time.time()
    
    # initializing the hidden state at the start of every epoch
    hidden = model.reset_states()
    
    for (batch, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions, hidden = model(inp, hidden)
              
              # reshaping the target because that's how the 
              # loss function expects it
              target = tf.reshape(target, (-1,))
              loss = loss_function(target, predictions)
              
          grads = tape.gradient(loss, model.variables)
          optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())

          if batch % 100 == 0:
              print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1,
                                                            batch,
                                                            loss))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.5975
Epoch 1 Batch 100 Loss 2.1609
Epoch 1 Batch 200 Loss 1.9387
Epoch 1 Loss 1.8163
Time taken for 1 epoch 1007.0127582550049 sec

Epoch 2 Batch 0 Loss 1.7427
Epoch 2 Batch 100 Loss 1.7149
Epoch 2 Batch 200 Loss 1.6851
Epoch 2 Loss 1.6786
Time taken for 1 epoch 1009.5401530265808 sec

Epoch 3 Batch 0 Loss 1.5864
Epoch 3 Batch 100 Loss 1.5735
Epoch 3 Batch 200 Loss 1.5432
Epoch 3 Loss 1.5330
Time taken for 1 epoch 1008.4457356929779 sec

Epoch 4 Batch 0 Loss 1.4756
Epoch 4 Batch 100 Loss 1.5105
Epoch 4 Batch 200 Loss 1.4949
Epoch 4 Loss 1.4980
Time taken for 1 epoch 1010.7342929840088 sec

Epoch 5 Batch 0 Loss 1.3859
Epoch 5 Batch 100 Loss 1.4656
Epoch 5 Batch 200 Loss 1.4051
Epoch 5 Loss 1.4314
Time taken for 1 epoch 1012.4246871471405 sec

Epoch 6 Batch 0 Loss 1.3024
Epoch 6 Batch 100 Loss 1.3982
Epoch 6 Batch 200 Loss 1.3920
Epoch 6 Loss 1.4050
Time taken for 1 epoch 1009.992424249649 sec

Epoch 7 Batch 0 Loss 1.2550
Epoch 7 Batch 100 Loss 1.3588
Epoch 7 Batch

In [38]:


# Evaluation step(generating text using the model learned)

# number of characters to generate
num_generate = 1000

# You can change the start string to experiment
start_string = 'The child'
# converting our start string to numbers(vectorizing!) 
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)

# empty string to store our results
text_generated = ''

# low temperatures results in more predictable text.
# higher temperatures results in more surprising text
# experiment to find the best setting
temperature = 0.97

# hidden state shape == (batch_size, number of rnn units); here batch size == 1
hidden = [tf.zeros((1, units))]
for i in range(num_generate):
    predictions, hidden = model(input_eval, hidden)

    # using a multinomial distribution to predict the word returned by the model
    predictions = predictions / temperature
    predicted_id = tf.multinomial(tf.exp(predictions), num_samples=1)[0][0].numpy()
    
    # We pass the predicted word as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    
    text_generated += idx2char[predicted_id]

print (start_string + text_generated)



The childhood rose that we may do,
And now the country seat her shall he died,
  And her shadow of regret?

If you can dress your head and stone be forget--
  That were coming home from the corner of her eye.

He did one that ever one to the summer of the rain!

She was wanting from the pine,
  And the song of his lower lay,
While I am sad and sent the truest,
  And one clothe lads that waits
  Of the black men doth clouds the stars,
      And the stars have broken the stars
      Stood on the brook, the bridge is passing through the starry skies.

    When the sun was clear, and the thing to be true
    A blaze in the stream of the steed,
      And the stars have broken the stairs,
    And the clearing and the long bright morning stair,
    And one that makes the dark bells they seemed to say:
     "But one song of the crowd.
            The stars come and the straw,
      And cold and still their welcome home.

    All the cold work that has lured the sea,
    And the clock stood cal